In [1]:
import numpy as np
import matplotlib as plt
import pandas as pd


In [2]:
dataset=pd.read_csv("CKD.csv")


In [3]:
dataset=pd.get_dummies(dataset,drop_first=True)


In [4]:
dataset=dataset.astype(int)

In [5]:
dataset

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2,76,3,0,148,57,3,137,4,12,...,0,0,0,0,0,0,1,1,0,1
1,3,76,2,0,148,22,0,137,4,10,...,1,0,0,0,0,0,1,0,0,1
2,4,76,1,0,99,23,0,138,4,12,...,1,0,0,0,0,0,1,0,0,1
3,5,76,1,0,148,16,0,138,3,8,...,1,0,0,0,0,0,1,0,1,1
4,5,50,0,0,148,25,0,137,4,11,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51,70,0,0,219,36,1,139,3,12,...,1,0,0,0,0,0,1,0,0,1
395,51,70,0,2,220,68,2,137,4,8,...,1,0,0,1,1,0,1,0,1,1
396,51,70,3,0,110,115,6,134,2,9,...,1,0,0,1,1,0,0,0,0,1
397,51,90,0,0,207,80,6,142,5,8,...,1,0,0,1,1,0,1,0,1,1


In [6]:
dataset.columns


Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [7]:
indep=dataset[["age","bp","sg_b","bgr","wc","rc"]]
dep=dataset['classification_yes']

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=0.30,random_state=0)

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid = {'solver':['newton-cg', 'lbfgs', 'liblinear', 'saga'],
             'penalty':['l2']} 



grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(x_train, y_train) 

Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\Users\Admin\anaconda3\Lib\site-packages\scipy\optimize\_linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\Admin\anaconda3\Lib\site-packages\scipy\optimize\_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga']},
             scoring='f1_weighted', verbose=3)

In [10]:
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(x_test) 

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
        

In [11]:
print(cm)

[[38  7]
 [ 8 67]]


In [12]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [13]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'penalty': 'l2', 'solver': 'newton-cg'}: 0.8752673500995648


In [14]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[38  7]
 [ 8 67]]


In [15]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.83      0.84      0.84        45
           1       0.91      0.89      0.90        75

    accuracy                           0.88       120
   macro avg       0.87      0.87      0.87       120
weighted avg       0.88      0.88      0.88       120



In [16]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])


0.9466666666666667

In [17]:
table=pd.DataFrame.from_dict(re)

In [18]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.178995,0.051605,0.008020,0.001442,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.927778,0.891667,0.874356,0.893717,0.944509,0.906405,0.025745,1
1,0.061658,0.014899,0.007117,0.001464,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.927778,0.839990,0.874356,0.893717,0.925465,0.892261,0.032921,3
2,0.008274,0.001679,0.005471,0.002862,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.927778,0.910254,0.816856,0.893717,0.944509,0.898623,0.044273,2
3,0.018875,0.003075,0.009720,0.003195,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.480769,0.480769,0.480769,0.480769,0.472319,0.479079,0.003380,4


In [19]:
age_input=float(input("Age:"))
bp_input=float(input("BMI:"))
wc_input=int(input("WC Yes 0 or 1:"))
sg_b_input=int(input("SU Yes 0 or 1:"))
rc_input=int(input("RU Yes 0 or 1:"))
classification_input=int(input("Classification Yes 0 or 1:"))



Age:25
BMI:89
WC Yes 0 or 1:1
SU Yes 0 or 1:1
RU Yes 0 or 1:1
Classification Yes 0 or 1:1


In [20]:
Future_Prediction=grid.predict([[age_input,bp_input,sg_b_input,wc_input,rc_input,classification_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]


C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
